In [1]:
pip install tensorflow


  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4847 sha256=2fe5bfc4d72d46330b4bf5507bec980906ba954872f539c08ab8f67e18a258b4
  Stored in directory: c:\users\melys\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built termcolor


In [8]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.7.0


## Chargement des données:

In [9]:
#Convertir les exemples de données d'entiers en nombres à virgule flottante
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [10]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(y_train)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
[5 0 4 ... 5 6 8]


Il y a 60 000 images dans l'ensemble d'apprentissage, chaque image étant représentée en 28 x 28 pixels.  
Chaque étiquette est un entier compris entre 0 et 9

## Construction du modèle:

Modèle par empilement de couches: Sequentialmodel est approprié pour une pile simple de couches où chaque couche a exactement un tenseur d'entrée et un tenseur de sortie(pas tres flexible).  
Layers : liste de couches à ajouter au modèle comme CONV2D et LSTM.

Les couches : 

* flatten (applatir) réduit simplement les vecteurs tenseurs, transforme le format à partir d' un tableau à deux dimensions (batch,) à un réseau à une dimension ([batch,1]).L'aplatissement ajoute une dimension de canal supplémentaire. Cette couche n'a pas de paramètres à apprendre ; elle reformate seulement les données.

* dropout couche dabondon : définit de manière aléatoire les unités d'entrée sur 0 avec une fréquence de rate à chaque étape pendant le temps d'entraînement -> ce qui permet d'éviter le surapprentissage. Les entrées non définies sur 0 sont augmentées de $ \frac{1}{1 - taux} $ de sorte que la somme de toutes les entrées reste inchangée.

* couche dense : est une couche neuronale densément connectée ou entièrement connectée, ou chaque neurone est connecté à tous les neurones de la couche précédente, elle consiste essentiellement en des poids (w) que nous multiplions par l'entrée, nous ajoutons des biais (b) et appliquons une fonction d'activation .

Fonctions d'activation: 
* ReLU : réseau très profond ou la charge de calcul est un problème, $ f(x)=max(0,x) $.
* Softmax : plus d'une seule sortie (classification à plus de 2 classes).Softmax convertit un vecteur de valeurs en une distribution de probabilité.  Les éléments du vecteur de sortie sont dans la plage (0, 1) et la somme à 1.  Chaque vecteur est géré indépendamment. L' axisargument définit sur quel axe de l'entrée la fonction est appliquée.    Softmax est souvent utilisé comme activation pour la dernière couche d'un réseau de classification car le résultat pourrait être interprété comme une distribution de probabilité.  le softmax de chaque vecteur x est calculé comme $ f(x)_i = \frac{\exp^{x_i}}{\sum_{k=0}^K \exp^{x_k}}$   C'est-à-dire que la composante i du vecteur $ f(x) $ est égale à l'exponentielle de la composante i du vecteur x divisée par la somme des exponentielles de toutes les composantes de x. 

* Sigmoid : une seule sortie entre 0 et 1 (classification binaire ), $ f(x) = \frac{\exp^{x}}{\exp^{x} + 1}$ 





In [11]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)), 
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])


In [12]:
#modèle retourne un vecteur de logits ou log-odds scores, un pour chaque classe.

predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.38991493, -0.10446891, -0.27904359, -0.0647276 ,  1.0073326 ,
        -0.36468804, -0.43663678, -0.05834304, -0.31406555, -0.7141819 ]],
      dtype=float32)

In [13]:
# vecteur de probabilité
tf.nn.softmax(predictions).numpy()

array([[0.14319237, 0.08733969, 0.07334912, 0.0908806 , 0.2654978 ,
        0.06732866, 0.06265461, 0.09146267, 0.07082474, 0.04746972]],
      dtype=float32)

## Compilation du modèle:

* Définir une fonction de perte : mesure qui évalue la précision ici on calcule la perte d'entropie croisée entre les étiquettes et les prédictions, qui prend un vecteur de logits et un True index et retourne une perte scalaire pour chaque exemple.  
Cette perte est égale au log de probabilité négatif de la vraie classe.   
La perte est nulle si le modèle est sûr de la bonne classe.

* Optimiseur adam : comment le modèle est mis à jour en fonction des données qu'il voit et sa fonction de perte, "adam" un algo de descente de gradient stochastique  
« efficace du point de vue informatique, nécessite peu de mémoire, est invariante au redimensionnement diagonal des gradients 
et convient bien aux problèmes volumineux en termes de données/paramètres ».  
* Liste des métriques à évaluer par le modèle lors de l'entraînement et des tests.

In [14]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [15]:
loss_fn(y_train[:1], predictions).numpy()

2.6981692

In [16]:
#compilation du modèle
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Entrainement du modèle:

Entraîner le modèle pour un nombre fixe d'époques (itérations sur un jeu de données) qui est une méthode pour ajuster vos paramètres du modèle et de minimiser la perte.


In [17]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2970 - accuracy: 0.9153
Epoch 2/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1457 - accuracy: 0.9560
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1082 - accuracy: 0.9678
Epoch 4/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0885 - accuracy: 0.9723
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0772 - accuracy: 0.9753


## Evaluation du modèle:

La méthode d'évaluation  vérifie les performances du modèles, généralement sur une « validation-set » ou « Test-set », et renvoie la valeur de perte et les valeurs des métriques pour le modèle en mode test.  
Les types de modes(verbose):
* verbose = 1, qui comprend à la fois une barre de progression et une ligne par époque (le mode par defaut).
* verbose = 0, signifie silencieux.
* verbose = 2, une ligne par époque, c'est-à-dire n° d'époque/n° total des époques.

In [18]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0763 - accuracy: 0.9771 - 536ms/epoch - 2ms/step


[0.07625770568847656, 0.9771000146865845]

In [19]:
#Si vous souhaitez que votre modèle renvoie une probabilité, 
#vous pouvez envelopper le modèle entraîné et lui attacher le softmax 
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [20]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.3172706e-07, 3.6173655e-09, 1.8448275e-06, 1.9058818e-05,
        2.1683927e-11, 8.7314561e-08, 3.3553360e-14, 9.9997449e-01,
        2.1785443e-08, 4.2398287e-06],
       [1.5309016e-08, 4.3805183e-05, 9.9990988e-01, 4.2988402e-05,
        1.3971094e-15, 1.4713569e-06, 1.8938621e-07, 2.4458601e-12,
        1.6908215e-06, 8.4900836e-14],
       [6.3323973e-06, 9.9708956e-01, 3.8307739e-04, 2.9523535e-05,
        8.5803622e-05, 5.0562234e-05, 5.3963635e-05, 1.0310108e-03,
        1.2698689e-03, 4.2644555e-07],
       [9.9995708e-01, 6.9490175e-10, 1.0611793e-05, 3.3243057e-07,
        5.2590809e-07, 8.4436615e-06, 1.0045597e-05, 7.0877313e-06,
        8.8360117e-08, 5.8305091e-06],
       [3.0610022e-06, 1.0824927e-09, 1.2345213e-06, 5.8230320e-08,
        9.9159509e-01, 1.6952828e-06, 6.7381848e-06, 3.0216164e-05,
        2.1402686e-06, 8.3598122e-03]], dtype=float32)>